In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
#import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
import time
from datetime import datetime, timedelta

all_currency_pairs = ['USDCHF10080',
                 'GBPUSD10080', 'EURUSD10080', 'USDJPY10080', 'USDCAD10080', 'AUDUSD10080', 'NZDUSD10080',
                 'GBPCHF10080', 'EURCHF10080', 'CHFJPY10080', 'CADCHF10080', 'AUDCHF10080', 'NZDCHF10080', 'EURGBP10080',
                 'GBPJPY10080', 'GBPCAD10080', 'GBPAUD10080', 'EURJPY10080', 'EURCAD10080', 'EURAUD10080', 'EURNZD10080',
                'CADJPY10080', 'AUDJPY10080', 'NZDJPY10080', 'AUDCAD10080', 'NZDCAD10080', 'AUDNZD10080'
                ]



headers = [ 'date', 'ignore', 'open', 'high', 'low', 'close', 'volume' ]

In [2]:
import os

for root, dirs, files in os.walk("files", topdown=False):
   pass



print(len(dirs))

360


In [3]:
available_weeks = dirs

In [4]:
all_time_next_week_trade =[]

for a in available_weeks:
    
    df = pd.read_csv('files/' + a + '/'+ 'next_week_trades.csv',  ) 
    
    #print(a,df)
    
    next_week_trades = []
    
    for index, row in df.iterrows():
        
        next_week_trades.append({'name': row.names,  'direction': row.next_week_prediction})
    
    
    if len(next_week_trades) > 0:
    
        all_time_next_week_trade.append({'week': a, 'next_week_trades':next_week_trades})
    
    
    
print(len(all_time_next_week_trade) )   
     

360


In [5]:
#loading 30 minutes data for all currency in to memeory to spead up processing

all_currency_thirty_minutes_data = {}

for z in all_currency_pairs:
    
    current_currency = z.replace('10080','')
    current_currency_df = pd.read_csv('../raw_30_minutes_data/' + current_currency + '30.csv', names=headers ) 
    
    all_currency_thirty_minutes_data[current_currency] = current_currency_df
    
    #print( current_currency, )
    
    
print(len(all_currency_thirty_minutes_data))






27


In [6]:
qualified_weeks =[]

for k in all_time_next_week_trade:
    
    current_week_data = k['next_week_trades']
    current_week_date = k['week']
    
    current_week_date = datetime.strptime(current_week_date, '%Y.%m.%d')

    week_start_date  = current_week_date + timedelta(days=7)
    week_end_date = current_week_date + timedelta(days=12)
    
    current_week_date = current_week_date.strftime("%Y.%m.%d")
    week_start_date = week_start_date.strftime("%Y.%m.%d")
    week_end_date = week_end_date.strftime("%Y.%m.%d")      

    
    

    #print(current_week_date, week_start_date, week_end_date)
    
    qualified_currencies = 0
   
    for l in current_week_data:
        
        
        current_currency = l['name'] 
        
        current_currency_df = all_currency_thirty_minutes_data[current_currency].copy()
        
        
        
        
        current_currency_df = current_currency_df.loc[  (current_currency_df['date'] >= week_start_date)
                                                   
                                                                                 & 
                                                              (current_currency_df['date'] <= week_end_date)
                                                            ]  
    
        
        
        
        
        
        current_currency_df = current_currency_df.reset_index(drop=True )
        
        #print( current_currency, week_start_date, week_end_date, current_currency_df)
       
        
       
        current_currency_dataframe_row_ammount = len(current_currency_df.index)
        
        if  current_currency_dataframe_row_ammount < 238:
            
            print(current_currency,len(current_currency_df),week_start_date,'less than 238, skipping ',current_week_date)
            break
            
            
            #raise Exception("lenght is lesss than 238")
            
            
        
        if  current_currency_dataframe_row_ammount != 240:
            
           
            
            if (current_currency_df['ignore'].tail(1).values == '22:30' ):
                
              
                
                last_row = current_currency_df.copy().tail(1)
                
                
                
                last_row.tail()['ignore'] = '23:00'
                
                
                current_currency_df = current_currency_df.append(last_row, ignore_index = True)
                
               
                
                
                last_row.tail()['ignore'] = '23:30'
                
                current_currency_df = current_currency_df.append(last_row, ignore_index = True)
                
              
                
                #print(current_currency, current_currency_df)
                
                if  len(current_currency_df.index) != 240:
              
                    print(current_currency, current_currency_df)
                    raise Exception("number of rows is still wrong after repair")
        
            else:
                
                print(current_currency,len(current_currency_df),week_start_date,"irrepairable,skipping ",current_week_date)
                break
        
        if current_currency_df['ignore'].iloc[-1] != '23:30' :
        
            print(current_currency, current_currency_df)
            raise Exception("last row is not 23:30")
        
        
      
        
        
       
        current_currency_df.to_csv("files/" + k['week'] + '/' + current_currency +".csv", index=None)
        
        qualified_currencies = qualified_currencies +1
        
        
        
    if qualified_currencies == len(current_week_data):
            
        qualified_weeks.append(k)
        
    
   
        
        #print(current_currency_df)
        
        
        
    #print('-----------------------------------------------------------------------------------------------', week_start_date)


print(len(qualified_weeks))


EURCHF 0 2015.01.11 less than 238, skipping  2015.01.04
AUDJPY 230 2015.03.15 less than 238, skipping  2015.03.08
EURCHF 184 2015.03.22 less than 238, skipping  2015.03.15
USDCHF 238 2015.05.31 irrepairable,skipping  2015.05.24
EURJPY 234 2015.06.28 less than 238, skipping  2015.06.21
USDCHF 182 2015.12.20 less than 238, skipping  2015.12.13
USDCHF 184 2015.12.27 less than 238, skipping  2015.12.20
CHFJPY 238 2016.01.03 irrepairable,skipping  2015.12.27
EURCHF 239 2016.01.10 irrepairable,skipping  2016.01.03
EURUSD 230 2016.03.20 less than 238, skipping  2016.03.13
NZDCHF 232 2016.05.01 less than 238, skipping  2016.04.24
NZDCHF 239 2016.09.11 irrepairable,skipping  2016.09.04
NZDCAD 239 2016.09.25 irrepairable,skipping  2016.09.18
USDCAD 237 2016.10.09 less than 238, skipping  2016.10.02
USDCHF 228 2016.12.25 less than 238, skipping  2016.12.18
NZDCHF 0 2017.03.12 less than 238, skipping  2017.03.05
GBPAUD 192 2017.06.04 less than 238, skipping  2017.05.28
NZDCHF 227 2017.08.27 less t

In [ ]:
timeframe = []
trade_values = []
unique_time_frames = []

for m in qualified_weeks:

    
    
    current_week = m['week']
   
    current_week_data = m['next_week_trades']
    
    current_week_drawdown_df = pd.DataFrame()
    
    current_week_profit = pd.DataFrame()
  

    
    
    for index, n in enumerate(current_week_data):
        
        
        
        current_currency = n['name']
        current_currency_direction = n['direction']
        current_currency_df =  pd.read_csv("files/" + current_week + '/' + current_currency +".csv" ) 
        
       
        
        
        # adding timeframe to the maximal draw down and profit df
        if index == 0:
            
            current_week_drawdown_df['date'] = current_currency_df['date']
            current_week_profit['date'] = current_currency_df['date']
           
            #saving unique_timeframes in a week to use later in the script
            if len(unique_time_frames) == 0:
                
                unique_time_frames = current_currency_df['ignore'].values
                
                
        
        
        #print(current_currency_df)
        maximal_draw_down =[]
        profit = []
        pips_corrector = 100000
            
        if current_currency.endswith('JPY'):
                
            pips_corrector = 1000
            
        
        
        
        for o in range(240):
                   
            if current_currency_direction == 'B':
                
                
                dd = current_currency_df.loc[o,'low'] - current_currency_df.loc[0,'open']
                p = current_currency_df.loc[o,'close'] - current_currency_df.loc[0,'open']
              
            if current_currency_direction == 'S':
                
                
                dd = current_currency_df.loc[0,'open'] - current_currency_df.loc[o,'high']
                p = current_currency_df.loc[0,'open'] - current_currency_df.loc[o,'close']
            
            
            maximal_draw_down.append(dd*pips_corrector)
            profit.append(p*pips_corrector)
                
        
        current_week_drawdown_df[current_currency] = maximal_draw_down
        current_week_profit[current_currency] = profit
            
    current_week_drawdown_df['empty'] = ['' for p in range(240)]      
    current_week_drawdown_df["sum"] = current_week_drawdown_df.drop(["empty", 'date'], axis=1, inplace=False).sum(axis=1)        
    
    current_week_drawdown_df.to_csv("files/" + current_week + '/' + 'maximal_draw_down.csv', index= None)
    
    
    
    current_week_profit['empty'] = ['' for p in range(240)]      
    current_week_profit["sum"] = current_week_profit.drop(["empty", 'date'], axis=1, inplace=False).sum(axis=1)        
    
    current_week_profit.to_csv("files/" + current_week + '/' + 'profit.csv', index= None)
    
    
    
    #print(current_week_drawdown_df)
        
    
    
    
    
    
    
    
    
    
    print('--------------------------------------------------------------------------------------------------', m)   
    
    
#print(current_week_drawdown_df)
#print(current_week_profit)

print(unique_time_frames)

-------------------------------------------------------------------------------------------------- {'week': '2015.01.11', 'next_week_trades': [{'name': 'GBPUSD', 'direction': 'S'}, {'name': 'EURUSD', 'direction': 'S'}, {'name': 'USDJPY', 'direction': 'S'}, {'name': 'USDCAD', 'direction': 'B'}, {'name': 'AUDUSD', 'direction': 'B'}, {'name': 'GBPCHF', 'direction': 'S'}, {'name': 'EURCHF', 'direction': 'S'}, {'name': 'CHFJPY', 'direction': 'S'}, {'name': 'CADCHF', 'direction': 'B'}, {'name': 'AUDCHF', 'direction': 'B'}, {'name': 'EURGBP', 'direction': 'S'}, {'name': 'GBPCAD', 'direction': 'B'}, {'name': 'CADJPY', 'direction': 'S'}, {'name': 'NZDJPY', 'direction': 'B'}, {'name': 'NZDCAD', 'direction': 'B'}, {'name': 'AUDNZD', 'direction': 'B'}]}
-------------------------------------------------------------------------------------------------- {'week': '2015.01.18', 'next_week_trades': [{'name': 'USDCHF', 'direction': 'B'}, {'name': 'GBPUSD', 'direction': 'B'}, {'name': 'EURUSD', 'direction

-------------------------------------------------------------------------------------------------- {'week': '2015.04.19', 'next_week_trades': [{'name': 'USDCHF', 'direction': 'B'}, {'name': 'EURUSD', 'direction': 'S'}, {'name': 'USDJPY', 'direction': 'B'}, {'name': 'NZDUSD', 'direction': 'S'}, {'name': 'GBPCHF', 'direction': 'B'}, {'name': 'EURCHF', 'direction': 'B'}, {'name': 'CHFJPY', 'direction': 'S'}, {'name': 'CADCHF', 'direction': 'B'}, {'name': 'EURGBP', 'direction': 'B'}, {'name': 'EURJPY', 'direction': 'S'}, {'name': 'EURAUD', 'direction': 'S'}, {'name': 'EURNZD', 'direction': 'S'}, {'name': 'NZDJPY', 'direction': 'S'}, {'name': 'NZDCAD', 'direction': 'B'}, {'name': 'AUDNZD', 'direction': 'B'}]}
-------------------------------------------------------------------------------------------------- {'week': '2015.04.26', 'next_week_trades': [{'name': 'EURUSD', 'direction': 'B'}, {'name': 'USDJPY', 'direction': 'S'}, {'name': 'GBPCHF', 'direction': 'S'}, {'name': 'EURCHF', 'direction

-------------------------------------------------------------------------------------------------- {'week': '2015.07.12', 'next_week_trades': [{'name': 'GBPUSD', 'direction': 'S'}, {'name': 'EURUSD', 'direction': 'B'}, {'name': 'USDJPY', 'direction': 'B'}, {'name': 'AUDUSD', 'direction': 'S'}, {'name': 'NZDUSD', 'direction': 'S'}, {'name': 'GBPCHF', 'direction': 'S'}, {'name': 'CHFJPY', 'direction': 'B'}, {'name': 'CADCHF', 'direction': 'S'}, {'name': 'AUDCHF', 'direction': 'S'}, {'name': 'EURGBP', 'direction': 'B'}, {'name': 'GBPAUD', 'direction': 'S'}, {'name': 'EURJPY', 'direction': 'B'}, {'name': 'EURCAD', 'direction': 'B'}, {'name': 'EURAUD', 'direction': 'B'}, {'name': 'CADJPY', 'direction': 'B'}, {'name': 'AUDJPY', 'direction': 'S'}, {'name': 'AUDCAD', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2015.07.19', 'next_week_trades': [{'name': 'GBPUSD', 'direction': 'B'}, {'name': 'EURUSD', 'direction

-------------------------------------------------------------------------------------------------- {'week': '2015.09.20', 'next_week_trades': [{'name': 'USDCHF', 'direction': 'S'}, {'name': 'GBPUSD', 'direction': 'B'}, {'name': 'EURUSD', 'direction': 'S'}, {'name': 'USDCAD', 'direction': 'B'}, {'name': 'AUDUSD', 'direction': 'B'}, {'name': 'NZDUSD', 'direction': 'S'}, {'name': 'GBPCHF', 'direction': 'B'}, {'name': 'EURCHF', 'direction': 'B'}, {'name': 'CHFJPY', 'direction': 'S'}, {'name': 'CADCHF', 'direction': 'S'}, {'name': 'AUDCHF', 'direction': 'B'}, {'name': 'EURGBP', 'direction': 'B'}, {'name': 'GBPJPY', 'direction': 'B'}, {'name': 'GBPCAD', 'direction': 'B'}, {'name': 'EURJPY', 'direction': 'B'}, {'name': 'EURCAD', 'direction': 'B'}, {'name': 'EURAUD', 'direction': 'S'}, {'name': 'EURNZD', 'direction': 'B'}, {'name': 'AUDJPY', 'direction': 'B'}, {'name': 'AUDCAD', 'direction': 'B'}, {'name': 'NZDCAD', 'direction': 'B'}, {'name': 'AUDNZD', 'direction': 'B'}]}
--------------------

-------------------------------------------------------------------------------------------------- {'week': '2015.11.29', 'next_week_trades': [{'name': 'USDCHF', 'direction': 'S'}, {'name': 'GBPUSD', 'direction': 'S'}, {'name': 'EURUSD', 'direction': 'S'}, {'name': 'USDJPY', 'direction': 'S'}, {'name': 'USDCAD', 'direction': 'S'}, {'name': 'AUDUSD', 'direction': 'B'}, {'name': 'NZDUSD', 'direction': 'B'}, {'name': 'GBPCHF', 'direction': 'B'}, {'name': 'EURCHF', 'direction': 'S'}, {'name': 'CHFJPY', 'direction': 'B'}, {'name': 'CADCHF', 'direction': 'B'}, {'name': 'AUDCHF', 'direction': 'B'}, {'name': 'NZDCHF', 'direction': 'B'}, {'name': 'EURGBP', 'direction': 'S'}, {'name': 'GBPAUD', 'direction': 'B'}, {'name': 'EURJPY', 'direction': 'S'}, {'name': 'EURCAD', 'direction': 'S'}, {'name': 'EURNZD', 'direction': 'S'}, {'name': 'NZDCAD', 'direction': 'B'}, {'name': 'AUDNZD', 'direction': 'S'}]}
------------------------------------------------------------------------------------------------

...

-------------------------------------------------------------------------------------------------- {'week': '2020.08.09', 'next_week_trades': [{'name': 'GBPUSD', 'direction': 'B'}, {'name': 'EURUSD', 'direction': 'B'}, {'name': 'USDCAD', 'direction': 'S'}, {'name': 'AUDUSD', 'direction': 'B'}, {'name': 'NZDUSD', 'direction': 'B'}, {'name': 'GBPCHF', 'direction': 'S'}, {'name': 'EURCHF', 'direction': 'B'}, {'name': 'CHFJPY', 'direction': 'B'}, {'name': 'AUDCHF', 'direction': 'S'}, {'name': 'EURGBP', 'direction': 'B'}, {'name': 'GBPJPY', 'direction': 'B'}, {'name': 'GBPAUD', 'direction': 'B'}, {'name': 'EURJPY', 'direction': 'B'}, {'name': 'CADJPY', 'direction': 'B'}, {'name': 'AUDJPY', 'direction': 'B'}, {'name': 'AUDCAD', 'direction': 'S'}, {'name': 'NZDCAD', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2020.08.16', 'next_week_trades': [{'name': 'GBPUSD', 'direction': 'S'}, {'name': 'EURUSD', 'direction

-------------------------------------------------------------------------------------------------- {'week': '2020.10.25', 'next_week_trades': [{'name': 'USDCHF', 'direction': 'S'}, {'name': 'GBPUSD', 'direction': 'B'}, {'name': 'EURUSD', 'direction': 'B'}, {'name': 'USDJPY', 'direction': 'S'}, {'name': 'AUDUSD', 'direction': 'B'}, {'name': 'CHFJPY', 'direction': 'B'}, {'name': 'CADCHF', 'direction': 'S'}, {'name': 'NZDCHF', 'direction': 'B'}, {'name': 'GBPJPY', 'direction': 'B'}, {'name': 'GBPCAD', 'direction': 'B'}, {'name': 'GBPAUD', 'direction': 'S'}, {'name': 'EURCAD', 'direction': 'B'}, {'name': 'CADJPY', 'direction': 'B'}, {'name': 'AUDCAD', 'direction': 'B'}, {'name': 'NZDCAD', 'direction': 'B'}, {'name': 'AUDNZD', 'direction': 'B'}]}
-------------------------------------------------------------------------------------------------- {'week': '2020.11.01', 'next_week_trades': [{'name': 'USDCHF', 'direction': 'B'}, {'name': 'GBPUSD', 'direction': 'S'}, {'name': 'EURUSD', 'direction

-------------------------------------------------------------------------------------------------- {'week': '2021.01.24', 'next_week_trades': [{'name': 'USDCHF', 'direction': 'S'}, {'name': 'GBPUSD', 'direction': 'B'}, {'name': 'EURUSD', 'direction': 'S'}, {'name': 'USDJPY', 'direction': 'S'}, {'name': 'USDCAD', 'direction': 'S'}, {'name': 'NZDUSD', 'direction': 'B'}, {'name': 'GBPCHF', 'direction': 'B'}, {'name': 'EURCHF', 'direction': 'S'}, {'name': 'CADCHF', 'direction': 'B'}, {'name': 'NZDCHF', 'direction': 'B'}, {'name': 'EURGBP', 'direction': 'S'}, {'name': 'GBPJPY', 'direction': 'B'}, {'name': 'GBPCAD', 'direction': 'B'}, {'name': 'GBPAUD', 'direction': 'S'}, {'name': 'EURCAD', 'direction': 'S'}, {'name': 'EURNZD', 'direction': 'S'}, {'name': 'CADJPY', 'direction': 'B'}, {'name': 'NZDJPY', 'direction': 'B'}, {'name': 'AUDCAD', 'direction': 'B'}, {'name': 'NZDCAD', 'direction': 'B'}, {'name': 'AUDNZD', 'direction': 'S'}]}
----------------------------------------------------------

-------------------------------------------------------------------------------------------------- {'week': '2021.04.04', 'next_week_trades': [{'name': 'USDCAD', 'direction': 'B'}, {'name': 'NZDUSD', 'direction': 'S'}, {'name': 'GBPCHF', 'direction': 'S'}, {'name': 'EURCHF', 'direction': 'B'}, {'name': 'CHFJPY', 'direction': 'S'}, {'name': 'AUDCHF', 'direction': 'B'}, {'name': 'GBPJPY', 'direction': 'S'}, {'name': 'GBPCAD', 'direction': 'S'}, {'name': 'GBPAUD', 'direction': 'B'}, {'name': 'EURJPY', 'direction': 'B'}, {'name': 'EURNZD', 'direction': 'S'}, {'name': 'AUDCAD', 'direction': 'S'}]}
-------------------------------------------------------------------------------------------------- {'week': '2021.04.11', 'next_week_trades': [{'name': 'USDJPY', 'direction': 'B'}, {'name': 'EURCHF', 'direction': 'B'}, {'name': 'CHFJPY', 'direction': 'B'}, {'name': 'AUDCHF', 'direction': 'B'}, {'name': 'EURGBP', 'direction': 'S'}, {'name': 'GBPCAD', 'direction': 'B'}, {'name': 'EURJPY', 'direction

-------------------------------------------------------------------------------------------------- {'week': '2021.07.11', 'next_week_trades': [{'name': 'USDCHF', 'direction': 'S'}, {'name': 'GBPUSD', 'direction': 'S'}, {'name': 'EURUSD', 'direction': 'S'}, {'name': 'USDJPY', 'direction': 'S'}, {'name': 'GBPCHF', 'direction': 'S'}, {'name': 'EURCHF', 'direction': 'S'}, {'name': 'CHFJPY', 'direction': 'S'}, {'name': 'AUDCHF', 'direction': 'S'}, {'name': 'GBPAUD', 'direction': 'B'}, {'name': 'EURJPY', 'direction': 'S'}, {'name': 'EURCAD', 'direction': 'S'}, {'name': 'EURAUD', 'direction': 'B'}, {'name': 'EURNZD', 'direction': 'S'}, {'name': 'CADJPY', 'direction': 'S'}, {'name': 'AUDJPY', 'direction': 'S'}, {'name': 'NZDJPY', 'direction': 'S'}, {'name': 'AUDCAD', 'direction': 'B'}, {'name': 'AUDNZD', 'direction': 'B'}]}


In [ ]:
day_time_frames = []

for index, w in enumerate(unique_time_frames):
    
    if index >= 192: 
        day = 'FRI'  
        day_time_frames.append(day + ' ' + w)
        
    elif index >= 144: 
        day = 'THUR'  
        day_time_frames.append(day + ' ' + w)
        
    elif index >=96: 
        day = 'WED'  
        day_time_frames.append(day + ' ' + w)
        
    elif index >=48: 
        day = 'TUE'  
        day_time_frames.append(day + ' ' + w)
        
    else: 
        day = 'MON'  
        day_time_frames.append(day + ' ' + w)
    
   
    
print(len(day_time_frames))

In [ ]:
#combining all maximal drawdowns for all weeks
combine_week_drawdown_df = pd.DataFrame()

combine_week_drawdown_df['date'] = day_time_frames

for index, q in enumerate(qualified_weeks):
    
    current_week = q['week']
    current_week_dd_df = pd.read_csv("files/" + current_week + '/' + 'maximal_draw_down' +".csv" ) 
    #print(current_week_dd_df)
    
   
    combine_week_drawdown_df[current_week] = current_week_dd_df['sum']
    

combine_week_drawdown_df['empty'] = ['' for r in range(240)]      
combine_week_drawdown_df["sum"] = combine_week_drawdown_df.drop(["empty",'date' ], axis=1, inplace=False).sum(axis=1)        

combine_week_drawdown_df.to_csv("files/" + 'combine_week_drawdown_df.csv', index= None)

print(combine_week_drawdown_df)

In [ ]:
#combining all profits drawdowns for all weeks
combine_week_profit_df = pd.DataFrame()

combine_week_profit_df['date'] = day_time_frames

for index, q in enumerate(qualified_weeks):
    
    current_week = q['week']
    current_week_profit_df = pd.read_csv("files/" + current_week + '/' + 'profit' +".csv" ) 
    #print(current_week_dd_df)
    
   
    combine_week_profit_df[current_week] = current_week_profit_df['sum']
    

combine_week_profit_df['empty'] = ['' for r in range(240)]      
combine_week_profit_df["sum"] = combine_week_profit_df.drop(["empty",'date' ], axis=1, inplace=False).sum(axis=1)        

combine_week_profit_df.to_csv("files/" + 'combine_week_profit_df.csv', index= None)

print(combine_week_profit_df)

In [ ]:
#loading all profit and drawdown dataframe to memory to speed up processing

qualified_weeks_dfs=[]

for v in qualified_weeks:
    
    current_week = v['week']

    current_week_profit_df = pd.read_csv("files/" + current_week + '/profit.csv')
    current_week_drawdown_df = pd.read_csv("files/" + current_week + '/maximal_draw_down.csv')
    
    qualified_weeks_dfs.append( 
                                { 'week': current_week,
                                   'profit_df' : current_week_profit_df,
                                   'drawdown_df': current_week_drawdown_df
                                }
                                )
    

print(qualified_weeks_dfs)
    
    

In [12]:
statistic_df = pd.DataFrame()

start_date_index_list =[]

cut_off_date_index_list =[]

draw_down_list = []

total_spread_list = []

profit_list=[]

profit_ratio_list=[]


#combine_week_drawdown_df = pd.read_csv("files/" + 'combine_week_drawdown_df.csv')
#combine_week_profit_df = pd.read_csv("files/" + 'combine_week_profit_df.csv')


#print(combine_week_drawdown_df)

start_date_index = [f for f in range(0, 239)]


draw_down = [f*-1 for f in range(100, 3000, 100)] 


total_spread = -200





for t in start_date_index:
    
    cut_off_date_index =   [f+1 for f in range( 1,(240-t)  ) ]
    
    #print(cut_off_date_index)
    
    for g in cut_off_date_index:
        
        
        for q in draw_down :
            
            
            strategy_profit = []
            
            current_maximal_drawdown = q + total_spread
            
            
            for s in qualified_weeks_dfs:
                
                current_week_calculated_profit = None
                
                current_drawdown_df = s['drawdown_df'].copy()
                current_drawdown_df = current_drawdown_df.iloc[t:]
              
                current_drawdown_df = current_drawdown_df.reset_index(drop=True)   
                current_drawdown_df = current_drawdown_df.iloc[:g]
                current_week_drawdown_list = current_drawdown_df['sum'].values
                
               
                
                

                current_profit_df = s['profit_df'].copy()
                current_profit_df = current_profit_df.iloc[t:]
                current_profit_df = current_profit_df.reset_index(drop=True)
                current_profit_df = current_profit_df.iloc[:g]
                current_week_profit_list = current_profit_df['sum'].values
                
                
                
                #print(current_week_profit_list)
                
            
                for r in range(len(current_week_drawdown_list)):
                    
                    
                    
                    current_drawdown = current_week_drawdown_list[r] - current_week_profit_list[0]
                    
                    current_profit = current_week_profit_list[r] - current_week_profit_list[0]
                    
                    
                    if r == 0:
                        #skipping the first row
                        continue
                    
                    
                    if current_drawdown <= q:
                        
                        # -1 signifies you have lost the weeks value of money
                        current_week_calculated_profit = -1
                        
                    
                        break
                        
                    
                    
                if current_week_calculated_profit == None:
                    
                    current_week_calculated_profit = ((current_profit + total_spread)/ (-1*current_maximal_drawdown) )
                    
                
                strategy_profit.append(current_week_calculated_profit)
                
            total_profit = sum(strategy_profit)
            
            profit_ratio = total_profit
            
            if profit_ratio > 50 :
                
                print( 'profit_ratio : ', profit_ratio)
                
                
            
            
            #print( 'start:', t+1, '  end:', g, '  drawdown:',q , '  profit:', total_profit, 'profit_ratio:',profit_ratio )
                
            start_date_index_list.append(t+1)

            cut_off_date_index_list.append(g)

            draw_down_list.append(q)

            total_spread_list.append(total_spread)

            profit_list.append(total_profit)

            profit_ratio_list.append(profit_ratio)
            
    print(t)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238


In [13]:
statistic_df['start_date_index'] = start_date_index_list


statistic_df['cut_off_date_index'] = cut_off_date_index_list

statistic_df['total_spread'] = total_spread_list

statistic_df['draw_down'] = draw_down_list





statistic_df['profit'] = profit_list

statistic_df['profit_ratio'] = profit_ratio_list





print(statistic_df)
statistic_df.to_csv('files/statistic_df.csv')

        start_date_index  cut_off_date_index  total_spread  draw_down  \
0                      1                   2          -200       -100   
1                      1                   2          -200       -200   
2                      1                   2          -200       -300   
3                      1                   2          -200       -400   
4                      1                   2          -200       -500   
...                  ...                 ...           ...        ...   
831715               239                   2          -200      -2500   
831716               239                   2          -200      -2600   
831717               239                   2          -200      -2700   
831718               239                   2          -200      -2800   
831719               239                   2          -200      -2900   

            profit  profit_ratio  
0      -321.000000   -321.000000  
1      -318.367500   -318.367500  
2      -318.494000